In [1]:
import os
import random
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


from torch.utils.data import DataLoader, TensorDataset

from Hyperparameters.Embeddings.BertTokenEmbedder import BertTokenEmbedder
from Hyperparameters.Dataloader.EmbeddingDataset import EmbeddingDataset
from Hyperparameters.Dataloader.collate_fn import collate_fn
from Hyperparameters.Models.BertPreTrainedClassifier import BertPreTrainedClassifier
from Hyperparameters.Training.ActiveLearningLoop import active_learning_loop
from Hyperparameters.Training.ActiveLearningLoop import query_entropy
from Hyperparameters.Models.CustomLoss import OrdinalLoss

from Hyperparameters.Utils.Misc import get_device

C:\Users\Bruce\anaconda3\envs\CIL\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Registering Model: SimpleModel (enforce_clean=False)
Git Info:
  User      : bbalfou <b_balfou@yahoo.com>
  Commit    : ba7bc29082098d103142682ee5eecebd634748d7
  Branch    : bruce
  File link : https://github.com/bennellis/CIL_Sentiment_analysis/blob/ba7bc29082098d103142682ee5eecebd634748d7/Hyperparameters\Models\ModelDummy.py


In [2]:

seed = 42

## Model Parameters

model_name="FacebookAI/roberta-base"
csv_path="data/Sentiment/training.csv"
lr = 1e-5
class_order = [0,1,2]
lr_top = 1e-5
lr_mid = 5e-6
lr_bot = 1e-6
dropout = 0.4

## Loss Parameters

temperature = 1.0
ce_weight = 0.2
margin = 0.0
use_cdw_ce = True
use_mae = False

## Old

# model_name="FacebookAI/roberta-base"
# csv_path="data/Sentiment/training.csv"
# seed = 42
# 
# lr = 1e-3
# class_order = [0,1,2]
# lr_top = 5e-5
# lr_mid = 3e-5
# lr_bot = 2e-5
# dropout = 0.4
# temperature = 0.5
# ce_weight = 0.1

In [3]:
df = pd.read_csv(csv_path, index_col=0)
label_map = {'negative': -1, 'neutral': 0, 'positive': 1}
df['label_encoded'] = df['label'].map(label_map)

In [4]:

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['sentence'], df['label_encoded'],
    stratify=df['label_encoded'], test_size=0.1, random_state=seed
)

In [5]:
embedder = BertTokenEmbedder(model_name)
features = embedder.fit_transform(df['sentence'].to_list())
labels = df['label_encoded'].to_numpy()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing batches:   0%|          | 0/3191 [00:00<?, ?it/s]

In [6]:


if embedder.is_variable_length:
    feature_dataset = EmbeddingDataset(features, labels)

    cache_name= model_name.replace("/", "_")
    cache_path = "cache/" + cache_name
    emb_dataset_path = cache_path + "emb_dataset.pt"


    if os.path.exists(emb_dataset_path):
        embedded_feature_dataset = torch.load(emb_dataset_path, weights_only=False)
    else:
        feature_dataloader = DataLoader(feature_dataset, batch_size=8,collate_fn=collate_fn)
        embedded_feature_dataset = embedder.embed_dataset(feature_dataloader)
        os.makedirs("cache", exist_ok=True)
        torch.save(embedded_feature_dataset, emb_dataset_path)

else:
    raise Exception("blaalalal")

In [7]:
model = BertPreTrainedClassifier(
    model_name = model_name,
    lr = lr,
    pt_lr_bot = lr_bot,
    pt_lr_mid = lr_mid,
    pt_lr_top = lr_top,
    class_order = class_order,
    ce_weight = ce_weight,
    temperature = temperature,
    frozen = True,
    custom_ll = True
)

Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:

criterion = OrdinalLoss(
    temperature=temperature,
    ce_weight = ce_weight,
    use_mae = use_mae,
    use_cdw_ce= use_cdw_ce,
    margin = margin
)
model.set_criterion(criterion=criterion)

[INFO] Using device: NVIDIA GeForce RTX 3080


In [9]:

active_learning_loop(
        model,
        get_device(),
        embedded_feature_dataset,
        query_entropy,
        max_rounds=1000,
        query_batch_size=1000,
        train_epochs_per_round=3,
        initial_label_count=1000,
        val_split=0.2,
        batch_size=32
)

[INFO] Using device: NVIDIA GeForce RTX 3080
1000, 80677, 1000


Training:   0%|          | 0/63 [00:00<?, ?batch/s]

Evaluating:   0%|          | 0/1277 [00:00<?, ?batch/s]

Training:   0%|          | 0/63 [00:00<?, ?batch/s]

Evaluating:   0%|          | 0/1277 [00:00<?, ?batch/s]

...

Getting Logits:   0%|          | 0/232 [00:00<?, ?batch/s]

Training:   0%|          | 0/3313 [00:00<?, ?batch/s]

Evaluating:   0%|          | 0/1277 [00:00<?, ?batch/s]

Training:   0%|          | 0/3313 [00:00<?, ?batch/s]

KeyboardInterrupt: 